        __  ______ _       ________  __        _ 
       / / / / __ \ |     / / __ \ \/ /  _    | |
      / /_/ / / / / | /| / / / / /\  /  (_)   / /
     / __  / /_/ /| |/ |/ / /_/ / / /  _     / / 
    /_/ /_/\____/ |__/|__/_____/ /_/  (_)  _/_/  
                                          /_/    
                                                      
               5168 7450 1701 5535     

                 author = Valerij Syroiezhin
               telegram = @NEU3RON
                  email = v.syroiezhin@gmail.com
                    url = https://github.com/syroiezhin

## ⚠️ The code below needs to be run in a file with a ".py" extension.

#### ✍🏻👂🏻 I want to use a simple option to convert voice to text, and there is no desire to look for other options, since the user will not notice a significant difference, and in any case, I will have to work hard with installing pyaudio! There is no way to avoid it.

In [ ]:
from speech_recognition import Microphone, Recognizer

def speech(source):
    sr = Recognizer()
    sr.pause_threshold = 0.5
    sr.adjust_for_ambient_noise(source=source, duration=0.5)
    audio_data = sr.listen(source=source)
    with open(f"audio.wav", "wb") as file: file.write(audio_data.get_wav_data()) # wav, raw, aiff, flac
    return sr.recognize_google(audio_data=audio_data, language='ru-RU').lower()

if __name__ == '__main__': 
    with Microphone() as source: print( speech(source) )

#### 🤖 There are different options for translating text into voice, but I need to write a program on Mac OS, and use it on Windows!
##### The first option, of course, is from Google, but there are difficulties with choosing a language, since there is no choice...
##### I don't have much time to reinvent my wheel.

In [ ]:
from gtts import gTTS
from os import path, system

if __name__ == '__main__':
    try:
        tts = gTTS('как твои дела, друг', lang='ru')
        if path.exists('gtts_0.mp3') == False: tts.save('gtts_0.mp3')
        else: print("True")
    except: pass
    finally: system('afplay gtts_0.mp3')

##### 🐵 The second option is more interesting, there are more voice acting options, but the voice of the robot is felt.

In [13]:
import pyttsx3
if __name__ == '__main__':
    try:
        tts = pyttsx3.init()
        tts.setProperty('voice', tts.getProperty('voices')[49].id) # Russian : 19,29,49
        tts.say("как твои дела, друг")
    except: pass
    finally: tts.runAndWait()

#### 👽 It turned out to be the coolest option with voice acting. He is so amazing. 
#### Although "there is no limit to perfection." 
#### Of course, there is room for improvement, 
#### but against the background of previous options, this one is promising.
#### I really liked the latest version of the library v3.1

In [ ]:
from sounddevice import play, wait
from torch import device, hub
from time import sleep

def say(speech):

    sr = 48000
    spd = int(sr * 4/5)  # in connection with the fast speech, Eugene decided to slow down his talk
    language = 'ru'      # Please study the library in the documentation :
    model_id = 'v3_1_ru' # github.com/snakers4/silero-models
    speaker = 'eugene'   # select character voice : aidar,eugene,random,baya,kseniya,xenia

    model, _ = hub.load( repo_or_dir = 'snakers4/silero-models' , model = 'silero_tts' , language = language , speaker = model_id )
    model.to(device('cpu'))
    audio = model.apply_tts( text = speech , speaker = speaker , sample_rate = sr )
    
    play( audio , spd )
    wait()

if __name__ == '__main__':
    speech = ' Пусть всегда будет солнце; Пусть всегда будет небо; Пусть всегда будет мама; Пусть всегда буду я '
    say(speech)

#### ⚠️ Before you start applying it on your macos - study the documentation and my manual on the project page!
# 📝 https://github.com/syroiezhin/got2help

### 🤔 A friend asked to make a function that from OGG format will receive a text version of the audio record 🔥

In [ ]:
from speech_recognition import Recognizer, AudioFile
from pydub import AudioSegment

def ogg2wav(NAME): AudioSegment.from_ogg(f'{NAME}.ogg').export(f"{NAME}.wav", format="wav")

def content(source, Recognizer): return Recognizer.recognize_google(Recognizer.record(source), language='ru-RU')

if __name__ == "__main__":
    NAME = 'audio'
    ogg2wav(NAME)
    with AudioFile(f"{NAME}.wav") as source: print( content( source, Recognizer() ) )

#### 👹 I'm creating a **repeated flight** that saves a recorded file and a file with the voice of the character Evgeniy from the SoundDevice library.

In [ ]:
from speech_recognition import Recognizer, Microphone, AudioFile
from sounddevice import playrec, wait
from torch import device, hub
from soundfile import write

def audio2text(recognizer, source): return recognizer.recognize_google( audio_data = recognizer.record(source) , language='ru-RU' )

def hear(recognizer, source, NAME):
    recognizer.pause_threshold = 1
    recognizer.adjust_for_ambient_noise( source = source , duration = 0.5 )
    audio_data = recognizer.listen(source)
    with open(f"{NAME}.wav", "wb") as file: file.write(audio_data.get_wav_data()) # raw, aiff, flac
    return recognizer.recognize_google( audio_data = audio_data , language = 'ru-RU' ).lower()

def say(speech, NAME):
    sr = 48000
    spd = int(sr * 4/5)
    language = 'ru'      # Please study the library in the documentation :
    model_id = 'v3_1_ru' # github.com/snakers4/silero-models
    speaker = 'eugene'   # select character voice : aidar,eugene,random,baya,kseniya,xenia
    model, _ = hub.load( repo_or_dir = 'snakers4/silero-models' , model = 'silero_tts' , language = language , speaker = model_id )
    model.to(device('cpu'))
    audio = model.apply_tts( text = speech , speaker = speaker , sample_rate = sr )
    rec = playrec( audio , spd , channels = 1 , blocking = True )
    wait()
    write( f'{NAME}.ogg', rec , spd )

if __name__ == "__main__":
    NAME = 'audio'
    recognizer = Recognizer()
    with Microphone() as source: heard = hear(recognizer, source, NAME)
    with AudioFile(f"{NAME}.wav") as source: content = audio2text(recognizer, source)
    print(content)
    say(heard, NAME)